### Part 2: change loss function from $loss = \frac{1}{n}\sum{(y_i - \hat(y_i))^2}$ to $loss = \frac{1}{n}\sum{|y_i - \hat{y_i}|}$, and using your mathmatical knowledge to get the right partial formual. Implemen the gradient descent code.

In [133]:
from sklearn.datasets import load_boston
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
from collections import defaultdict
import math
import time
import heapq as hq
%matplotlib inline

In [155]:
data = load_boston()
X, y = data['data'], data['target']

In [156]:
def get_prices(k, b, x):
    return np.dot(x, k) + b

def get_loss(y, y_hat): # to evaluate the performance 
    return sum(abs(y_i - y_hat_i) for y_i, y_hat_i in zip(list(y), list(y_hat))) / len(list(y))

def partial_k(x, y, y_hat):
    n = len(y)
    gradient = 0
    for x, y, y_hat in zip(list(x), list(y), list(y_hat)):
        if y > y_hat:
            gradient += x/n
        elif y < y_hat:
            gradient -= x/n
    return -gradient

def partial_b(x, y, y_hat):
    n = len(y)
    gradient = 0
    for x, y, y_hat in zip(list(x), list(y), list(y_hat)):
        if y > y_hat:
            gradient += 1/n
        elif y < y_hat:
            gradient -= 1/n
    
    return -gradient

In [175]:
def train_by_gradient_descend(trying_times, learning_rate):

    k = best_supervised_k = [random.random() * 200 - 100 for i in range(13)]   #Why??
    b = best_supervised_b = random.random() * 200 - 100

    min_loss = float('inf') 

    for i in range(trying_times):
        y_hat = get_prices(k, b, X)
        k -= partial_k(X, y, y_hat) * learning_rate
        b -= partial_b(X, y, y_hat) * learning_rate
        loss = get_loss(y, y_hat)
        if i % 5000 == 0:
            print(loss)
        if loss < min_loss:
            min_loss = loss
            best_directed_k = k
            best_directed_b = b
            if i % 50 == 0: 
                print('When time is : {}, get best_k: {} best_b: {}, and the loss is: {}'.format(i, best_directed_k, best_directed_b, min_loss))
    

In [177]:
train_by_gradient_descend(100000, 0.01)

7035.09914848253
When time is : 0, get best_k: [-67.08355173 -51.99449125  20.71442417  94.10192458  20.37826749
 -62.30188403 -96.43973009  84.26127783 -20.39227809  -2.52347789
  59.20166413   9.26367348  -3.74728826] best_b: 70.00012692214668, and the loss is: 7035.09914848253
When time is : 50, get best_k: [-66.13207012 -52.41490627  21.63962733  94.10789296  20.40792438
 -62.15581834 -86.94951665  84.01262563 -19.9856773    9.62719405
  59.51747045   5.2074719   -2.35639261] best_b: 70.02269609210717, and the loss is: 1618.7295343651156
When time is : 100, get best_k: [-65.45805744 -51.75423434  22.13148069  94.11480996  20.43077701
 -61.98736103 -78.64231507  83.86838079 -20.24733738   8.65094899
  59.76234002   4.44112487  -1.30842462] best_b: 70.04593719882654, and the loss is: 1474.6771825697897
When time is : 150, get best_k: [-64.74924032 -50.87913552  22.61241705  94.12172696  20.4529241
 -61.8241951  -70.49884076  83.72429614 -20.49646781   7.78201618
  59.98565622   3.944

In [ ]:
def partial_k(x, y, y_hat):
    n = len(y)
    gradient = 0
    for x, y, y_hat in zip(list(x), list(y), list(y_hat)):
        if y > y_hat:
            gradient += x/n
        elif y < y_hat:
            gradient -= x/n
    return -gradient

def partial_b(x, y, y_hat):
    n = len(y)
    gradient = 0
    for x, y, y_hat in zip(list(x), list(y), list(y_hat)):
        if y > y_hat:
            gradient += 1/n
        elif y < y_hat:
            gradient -= 1/n
    
    return -gradient

### Part 3: Finish the Solution Parse Part of Edit-Distance

In [46]:
solution = {}

In [47]:
from functools import lru_cache

In [38]:
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    candidates = [
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),  # string 1 delete tail
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),  # string 1 add tail of string2
    ]
    
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))

    candidates.append(both_forward)
    
    min_distance, operation = min(candidates, key=lambda x: x[0])
    
    solution[(string1, string2)] = operation 
    
    return min_distance

In [62]:
def edit_distance(string1, string2):
    global solution
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)

    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    a = (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1))               # 删除
    b = (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2))               #插入
    candidates = [
        a,  # string 1 delete tail
        b,  # string 1 add tail of string2
    ]

    if tail_s1 == tail_s2:
        c = edit_distance(string1[:-1], string2[:-1]) + 0
        both_forward = (c, '')
    else:
        d = edit_distance(string1[:-1], string2[:-1]) + 1
        both_forward = (d, 'SUB {} => {}'.format(tail_s1, tail_s2))

    candidates.append(both_forward)

    min_distance, operation = min(candidates, key=lambda x: x[0])
    
    #print(sorted(candidates, key=lambda x: x[0]))

    solution[(string1, string2)] = operation

    return min_distance

In [81]:
def parse_solution(string1, string2):

    if not string1:
        return []

    k = (string1, string2)
    res = parse_solution(string1[:-1], string2) + [{k:solution[(string1, string2)]}]
   
    #res = parse_solution(string1[:-1], string2[:-1]) + [{k:solution[(string1, string2)]}]

    return res

In [82]:
string1 = 'be'
string2 = 'beg'
edit_distance(string1, string2)
for k, v in solution.items():
    print(k, ': ', v)

[(0, ''), (2, 'DEL b'), (2, 'ADD b')]
[(1, 'ADD e'), (2, 'SUB b => e'), (3, 'DEL b')]
[(2, 'ADD g'), (3, 'SUB b => g'), (4, 'DEL b')]
[(0, ''), (2, 'DEL b'), (2, 'ADD b')]
[(1, 'ADD e'), (2, 'SUB b => e'), (3, 'DEL b')]
[(0, ''), (2, 'DEL b'), (2, 'ADD b')]
[(1, 'DEL e'), (2, 'SUB e => b'), (3, 'ADD b')]
[(0, ''), (2, 'DEL b'), (2, 'ADD b')]
[(0, ''), (2, 'DEL e'), (2, 'ADD e')]
[(0, ''), (2, 'DEL b'), (2, 'ADD b')]
[(1, 'ADD e'), (2, 'SUB b => e'), (3, 'DEL b')]
[(1, 'ADD g'), (2, 'SUB e => g'), (3, 'DEL e')]
('b', 'b') :  
('b', 'be') :  ADD e
('b', 'beg') :  ADD g
('be', 'b') :  DEL e
('be', 'be') :  
('be', 'beg') :  ADD g


In [83]:
print(parse_solution(string1, string2))     #有问题，漏掉了删除操作

[{('b', 'be'): 'ADD e'}, {('be', 'beg'): 'ADD g'}]


In [84]:
edit_distance('be', 'beg')

[(0, ''), (2, 'DEL b'), (2, 'ADD b')]
[(1, 'ADD e'), (2, 'SUB b => e'), (3, 'DEL b')]
[(2, 'ADD g'), (3, 'SUB b => g'), (4, 'DEL b')]
[(0, ''), (2, 'DEL b'), (2, 'ADD b')]
[(1, 'ADD e'), (2, 'SUB b => e'), (3, 'DEL b')]
[(0, ''), (2, 'DEL b'), (2, 'ADD b')]
[(1, 'DEL e'), (2, 'SUB e => b'), (3, 'ADD b')]
[(0, ''), (2, 'DEL b'), (2, 'ADD b')]
[(0, ''), (2, 'DEL e'), (2, 'ADD e')]
[(0, ''), (2, 'DEL b'), (2, 'ADD b')]
[(1, 'ADD e'), (2, 'SUB b => e'), (3, 'DEL b')]
[(1, 'ADD g'), (2, 'SUB e => g'), (3, 'DEL e')]


1

In [80]:
solution

{('b', 'b'): '',
 ('b', 'be'): 'ADD e',
 ('b', 'beg'): 'ADD g',
 ('be', 'b'): 'DEL e',
 ('be', 'be'): '',
 ('be', 'beg'): 'ADD g'}

In [40]:
edit_distance('ABCDE', 'ABCCEF')

2

In [41]:
solution

{('A', 'A'): '',
 ('A', 'AB'): 'ADD B',
 ('A', 'ABC'): 'ADD C',
 ('A', 'ABCC'): 'ADD C',
 ('A', 'ABCCE'): 'ADD E',
 ('A', 'ABCCEF'): 'ADD F',
 ('AB', 'A'): 'DEL B',
 ('AB', 'AB'): '',
 ('AB', 'ABC'): 'ADD C',
 ('AB', 'ABCC'): 'ADD C',
 ('AB', 'ABCCE'): 'ADD E',
 ('AB', 'ABCCEF'): 'ADD F',
 ('ABC', 'A'): 'DEL C',
 ('ABC', 'AB'): 'DEL C',
 ('ABC', 'ABC'): '',
 ('ABC', 'ABCC'): 'ADD C',
 ('ABC', 'ABCCE'): 'ADD E',
 ('ABC', 'ABCCEF'): 'ADD F',
 ('ABCD', 'A'): 'DEL D',
 ('ABCD', 'AB'): 'DEL D',
 ('ABCD', 'ABC'): 'DEL D',
 ('ABCD', 'ABCC'): 'SUB D => C',
 ('ABCD', 'ABCCE'): 'ADD E',
 ('ABCD', 'ABCCEF'): 'ADD F',
 ('ABCDE', 'A'): 'DEL E',
 ('ABCDE', 'AB'): 'DEL E',
 ('ABCDE', 'ABC'): 'DEL E',
 ('ABCDE', 'ABCC'): 'DEL E',
 ('ABCDE', 'ABCCE'): '',
 ('ABCDE', 'ABCCEF'): 'ADD F'}

In [42]:
print(parse_solution('ABCDE', 'ABCCEF'))

[{('A', 'ABCCEF'): 'ADD F'}, {('AB', 'ABCCEF'): 'ADD F'}, {('ABC', 'ABCCEF'): 'ADD F'}, {('ABCD', 'ABCCEF'): 'ADD F'}, {('ABCDE', 'ABCCEF'): 'ADD F'}]


# note：

In [ ]:
细节及矩阵转化计算，参见 https://www.jianshu.com/p/a617d20162cf
'''
dp[i][j]表示word1[0...i-1]到word2[0...j-1]的编辑距离。
而dp[i][0]显然等于i <= 只需要做i次删除操作就可以了。
同理dp[0][i]等于i，<= 只需做i次插入操作就可以了。

dp[i-1][j]变到dp[i][j]需要加1，因为word1[0...i-2]到word2[0...j-1]的距离是dp[i-1][j]，
而word1[0...i-1]到word1[0...i-2]需要执行一次删除，所以dp[i][j]=dp[i-1][j]+1；

同理dp[i][j]=dp[i][j-1]+1，因为还需要加一次word2的插入操作。

如果word[i-1]==word[j-1]，则dp[i][j]=dp[i-1][j-1]，
如果word[i-1]!=word[j-1]，那么需要执行一次替换replace操作，所以dp[i][j]=dp[i-1][j-1]+1，

'''
'''
编辑距离主要分三个操作   插入、删除、替换 
len(str1) = m  len(str2) = n
dp[m][n] 表示从str1到str2的编辑距离 
dp[m][n] = dp[m-1][n] + 1  对应删除
dp[m][n] =dp[m][n-1] + 1   对应插入
if str1[m - 1] == str2[n-1] 最后一项相同
    dp[m][n] = dp[m-1][n-1]
else
    dp[m][n] = dp[m-1][n-1] 对应替换
转化为函数就是老师写的上面的那个方法
'''
'''
class Solution:
    # @return an integer
    def minDistance(self, word1, word2):
        m=len(word1)+1; n=len(word2)+1
        dp = [[0 for i in range(n)] for j in range(m)]
        for i in range(n):
            dp[0][i]=i
        for i in range(m):
            dp[i][0]=i
        for i in range(1,m):
            for j in range(1,n):
                dp[i][j]=min(dp[i-1][j]+1, dp[i][j-1]+1, dp[i-1][j-1]+(0 if word1[i-1]==word2[j-1] else 1))
        return dp[m-1][n-1]
'''


## 参考CSN6666

In [85]:
class edit_distance():
    '''
    compute是老师例子里面的“edit_distance”
    '''
    def __init__(self, string1, string2, solution = {}):
        self.solution = {}
        self.parsed_solution = []
        self.string1 = string1
        self.string2 = string2
        self.string_used = False
    
    @lru_cache(maxsize=2**10)
    def compute(self, string1 = '', string2 = ''):
        if self.string_used == False:
            string1 = self.string1
            string2 = self.string2
            self.string_used = True
        if len(string1) == 0: 
            self.solution[(string1, string2)] = 'ADD {}'.format(string2)
            self.string_used = False
            return len(string2)
        if len(string2) == 0: 
            self.solution[(string1, string2)] = 'DEL {}'.format(string1)
            self.string_used = False
            return len(string1)

        tail_s1 = string1[-1]
        tail_s2 = string2[-1]

        candidates = [
            (self.compute(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),  # string 1 delete tail
            (self.compute(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),  # string 1 add tail of string2
        ]

        if tail_s1 == tail_s2:
            both_forward = (self.compute(string1[:-1], string2[:-1]) + 0, '')
        else:
            both_forward = (self.compute(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))

        candidates.append(both_forward)

        min_distance, operation = min(candidates, key=lambda x: x[0])

        self.solution[(string1, string2)] = operation 

        return min_distance
    
    @lru_cache(maxsize=2**10)
    def parse_solution(self, string1 = '', string2 = ''):
        if self.string_used == False:
            string1 = self.string1
            string2 = self.string2
            self.string_used = True
            
        operation = self.solution[string1, string2]
        if len(string1) == 0: 
            self.string_used = False
            self.parsed_solution.append(operation + ' before index={}'.format(len(string1)))
            return self.parsed_solution[::-1]
        if len(string2) == 0: 
            self.string_used = False
            self.parsed_solution.append(operation + ' before index={}'.format(len(string1)))
            return self.parsed_solution[::-1]
        
        if operation == '':
            self.parse_solution(string1[:-1], string2[:-1])
        else:
            operator = operation[:3]
            operand = operation[3:]
            if operator == 'ADD':
                self.parse_solution(string1, string2[:-1])
                self.parsed_solution.append(operation + ' after index={}'.format(len(string1)-1))
            elif operator == 'DEL':
                self.parse_solution(string1[:-1], string2)
                self.parsed_solution.append(operation + ' at index={}'.format(len(string1)-1))
            elif operator == 'SUB':
                self.parse_solution(string1[:-1], string2[:-1])
                self.parsed_solution.append(operation + ' at index={}'.format(len(string1)-1))
        return self.parsed_solution[::-1]

In [87]:
distance = edit_distance('see you','nice to meet you')
distance.compute()
distance.parse_solution()

['ADD t after index=2',
 'ADD e after index=2',
 'ADD m after index=1',
 'ADD   after index=1',
 'ADD o after index=1',
 'ADD t after index=1',
 'ADD   after index=1',
 'SUB s => c at index=0',
 'ADD ni before index=0']

## Part 5-1: review machine learning

1. Why do we use Derivative / Gredient to fit a target function?¶

Ans:能够使变量沿着正确的方向变化，提高拟合效率

2. In the words 'Gredient Descent', what's the Gredient and what's the Descent?¶

Ans:梯度是损失函数对参数的偏导数，下降即是按梯度的负方向；
梯度是高维导数的另一种说法，它给出了最陡峭的上升方向，对于二次曲线，就是对x的导数，对于三维曲面，梯度就是曲面某点对指定方向的偏导数；
    由于曲面上某一点有无数个方向，每个方向的梯度都不同，而只有一个使函数上升最快的方向，这个方向的梯度绝对值最大，与之相反的方向就是下降最快的方向。
    更抽象化而言，梯度是函数在某一空间点的方向导数，梯度下降是指沿着这个方向函数的值下降的最快

3. What's the advantages of the 3rd gradient descent method compared to the previous methods?

Ans:法1 随机产生k，b，其值完全没有反向，最终loss的最小值完全是随机的；法2 有简单的监督学习模式，且针对k、b定义了四个变化方向，并选择loss减小的方向变化，直至loss不能减小，但k、b变化方向具有随机性；法3 采用梯度下降的方法，可使得k，b沿loss值减小的方向变化，提高了拟合效率

4. Using the simple words to describe: What's the machine leanring.¶

Ans:机器学习是一种通过各种手段，将人类思维体系和知识积累映射进机器的方法或技术，达到人机交融的效果。
简单的从函数角度，即 输入x,y及学习方法，输出x与y之间对应关系；主要采用数学及相关理论构建算法，但涉及各个学科。

### Part 5: Answer following questions: 

1. Why do we need dynamic programming? What's the difference of dynamic programming and previous talked `search` problme? 

一般搜索问题中，细分的子问题经常会有交叠，即出现重复的子问题，而随着数据量的增多，过多的子问题会显著提高算法的时间复杂度

2. Why do we still need dynamic programming? Why not we train a machine learning to fit a function which could get the `right` answer based on inputs?

首先，动态规划和机器学习并不冲突，而是有着各自适用的范围；在有些情况，使用动态规划很方便，可以避免训练机器需要的庞大数据及不断的算法优化，如在路径搜索中使用广度优先并通过动态规划来优化，很好的解决了这个问题，而采用机器学习，会更复杂，涉及模型设置、大量数据及数据特征等；而在另一些情况下，机器学习更能发挥作用，如垃圾邮件的识别及处理等。


3. Can you catch up at least 3 problems which could solved by Dynamic Programming? 

（快递送餐）最短路径  凑零钱问题   

4. Can you catch up at least 3 problems wich could sloved by Edit Distance? 

文本相似度  单词纠错   论文查重  基因序列比对

5. Please summarize the three main features of Dynamic Programming, and make a concise explain for each feature. 

一般而言，使用动态规划的问题具有以下三个特点：
    (1)有重叠子问题，且子问题相互不是独立的，一个子问题可能在下个决策中多次用到； 
    (2)无后效性：即某阶段状态一旦确定，就不受这个状态以后决策的影响。
    (3) 最优化原理：如果问题的最优解所包含的子问题的解也是最优的，就称该问题具有最优子结构，即满足最优化原理。

6. What's the disadvantages of Dynamic Programming? (You may need search by yourself in Internet)

（1）没有统一的标准模型；找出子问题的表现方式需要创造力和实验，经常需要对每类问题进行具体分析，非熟练的分析人员难以准确对原问题进行合理分解，导致应用上的局限。
（2）数值方法求解时存在维数灾；状态空间可能呈指数增长。如果一维状态变量有m个取值，则n维问题的状态就有mn个值。对于n较大的实际问题，其计算成本上无法容忍。